## Recogninsing human actions using 3D CNN

Recognizing human actions is one of most popular computer vision method which finds mutiple applications in lots of fields such as video surveillance, customer attributes, shopping behavior analysis.

In our final project, we consider the automated recognition of human actions in some videos. We proposed to build up a 3D CNN model for action recognition. In order to capture motion information from multiple adjacent frames, we proposed to extract features from both spatial and temporal dimensions. Based on this feature extractor, a 3D convolutional neural network will be built up. This CNN will generates multiple channels of information and performs convolution and subsampling separately. The final feature representation is obtained by conbining information from all channels.

## Implementation details
---
Completed tasks
1. Implemented for the UFC dataset 
2. Implemented regularisation using SIFT and MEHI features
3. Used a Hardwired layer that extracts features from frames
4. Implemented 5 CNN models with similar structure but different kernel size
5. Tried ensembles of this 4, different combinations and compared the results
6. Used swift activation function for better results





# Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16
# #Installing these libraries is important since, the newer versions have removed these libraries

     |████████████████████████████████| 25.0MB 122kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 76kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
# Import standard and supportive libraries
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
from tensorflow.python.client import device_lib
from tensorflow import keras
# from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
import pickle

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print("Available devices for trainning:", get_available_devices())

Available devices for trainning: ['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


In [ ]:
#Try to use tensorflow in GPU
config_tf = tf.compat.v1.ConfigProto(log_device_placement=True)
config_tf.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config_tf)
tf.config.run_functions_eagerly(True)

# Import models and layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers.core import Dense, Flatten, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D

# Import utilities
from keras.optimizers import SGD,RMSprop
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils, generic_utils
from tensorflow.compat.v1.keras.backend import set_session
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

set_session(session)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7



# LAYER 1, HARDWIRED LAYER

In [ ]:
def H1_layer(file_id):

  inflation = 1
  # image attributes
  #80, 60 is the Image dimensions and 9 is the number of frames
  img_r, img_c, img_d = 80, 60, 9

  capture = cv2.VideoCapture(file_id)
  fps = capture.get(cv2.CAP_PROP_FPS)
  # print("Frames per second using video.get(cv2.CAP_PROP_FPS): {0}".format(fps))
  for j in range(inflation):
      frame_list = []
      grey_list = []
      horz_grad_list = []
      vert_grad_list = []
      horz_flow_list = []
      vert_flow_list = []

      flow_calc = False
      prvs = None
      for i in range(img_d):
          # print(i)
          success, frame = capture.read()
          frame = cv2.resize(frame,(img_r,img_c),interpolation=cv2.INTER_AREA)
          gray =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

          sobelx64f = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize= -1)
          abs_sobel64f = np.absolute(sobelx64f)
          vert_grad = np.uint8(abs_sobel64f)

          sobelx64f = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize= -1)
          abs_sobel64f = np.absolute(sobelx64f)
          horz_grad = np.uint8(abs_sobel64f)

          next = gray
          if (flow_calc):
            hsv = np.zeros_like(prvs)
            hsv[...,1] = 255
            # ret, frame2 = capture.read()
            # next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            horz_flow = cv2.normalize(flow[...,0], None, 0, 255, cv2.NORM_MINMAX)     
            vert_flow = cv2.normalize(flow[...,1], None, 0, 255, cv2.NORM_MINMAX)
            horz_flow = horz_flow.astype('uint8')
            vert_flow = vert_flow.astype('uint8')

            horz_flow_list.append(horz_flow)
            vert_flow_list.append(vert_flow)
          
          flow_calc = True
          prvs = gray

          frame_list.append(frame)
          grey_list.append(gray)
          horz_grad_list.append(horz_grad)
          vert_grad_list.append(vert_grad)
          

      cv2.destroyAllWindows()
      # ipt = np.asarray(frame_list)
      # ipt = np.rollaxis(np.rollaxis(ipt,2,0),2,1)

      ipt_g = np.asarray(grey_list)
      ipt_g = np.rollaxis(ipt_g,2,1)
      ipt = ipt_g

      ipt_hg = np.asarray(horz_grad_list)
      ipt_hg = np.rollaxis(ipt_hg,2,1)

      ipt_vg = np.asarray(vert_grad_list)
      ipt_vg = np.rollaxis(ipt_vg,2,1)

      ipt_hf = np.asarray(horz_flow_list)
      ipt_hf = np.rollaxis(ipt_hf,2,1)

      ipt_vf = np.asarray(vert_flow_list)
      ipt_vf = np.rollaxis(ipt_vf,2,1)

      # ipt_g = np.concatenate((ipt_g, ipt_hg))
      # ipt_g = np.concatenate((ipt_g, ipt_vg))
      # ipt_g = np.concatenate((ipt_g, ipt_hf))
      # ipt_g = np.concatenate((ipt_g, ipt_vf))

      # Training_set.append(ipt)
      num_feature_maps = np.array([ipt.shape[0],ipt_hg.shape[0],ipt_vg.shape[0],ipt_hf.shape[0],ipt_vf.shape[0]])
      # print(ipt_g.shape,ipt.shape,ipt_hg.shape,ipt_vg.shape,ipt_hf.shape,ipt_vf.shape)
  capture.release()

  return ipt_g,ipt_hg,ipt_vg,ipt_hf,ipt_vf


In [ ]:
# Training_set = {}
def apply_H1(dirname,split_val=0.2,KTH=True,num_frames=9):
  if (KTH):
    actions = ['boxing','handclapping','handwaving','jogging','running','walking']
  else:
    actions = ["ApplyEyeMakeup","Basketball","Diving","HorseRiding","PizzaTossing","Rowing"]
  Training_set = {}
  Training_set['M1'] = []
  Training_set['M2'] = []
  Training_set['M3'] = []
  Training_set['M4'] = []
  Training_set['M5'] = []
  Train_y = []
  Test_set = {}
  Test_set['M1'] = []
  Test_set['M2'] = []
  Test_set['M3'] = []
  Test_set['M4'] = []
  Test_set['M5'] = []
  Test_y = []
  action_class = 0
  for action in actions:
    raw_input_dir = os.listdir(dirname+action)
    print(action,'loading...')
    total_samples = len(raw_input_dir)
    test_samples = int(total_samples*split_val)
    test_counter = 0
    for file_name in raw_input_dir:
      file_id = dirname + action + '/' + file_name
      m1,m2,m3,m4,m5 = H1_layer(file_id)
      if (test_counter < test_samples):
        Test_set['M1'].append(m1)
        Test_set['M2'].append(m2)
        Test_set['M3'].append(m3)
        Test_set['M4'].append(m4)
        Test_set['M5'].append(m5)
        Test_y.append(action_class)
        test_counter = test_counter + 1
        # print('added in test set',test_counter,test_samples,total_samples)
      else:
        Training_set['M1'].append(m1)
        Training_set['M2'].append(m2)
        Training_set['M3'].append(m3)
        Training_set['M4'].append(m4)
        Training_set['M5'].append(m5)
        Train_y.append(action_class)
    print(action,'loaded')
    action_class = action_class + 1
  # Training_set = np.array(Training_set)
  return Training_set,Test_set,Train_y,Test_y

## Preprocessing the channels

Resulting channels obtained are M1,M2,M3,M4,M5 are processed and reshaped

In [ ]:
def provide_channels(Training_set):
  M1_set = np.array(Training_set['M1'])
  M2_set = np.array(Training_set['M2'])
  M3_set = np.array(Training_set['M3'])
  M4_set = np.array(Training_set['M4'])
  M5_set = np.array(Training_set['M5'])
  # print("The resulting dataset shapes")
  # print(M1_set.shape,M2_set.shape,M3_set.shape,M4_set.shape,M5_set.shape)

  M1_set = M1_set.astype('float32')
  M1_set -= np.mean(M1_set)
  M1_set /= np.max(M1_set)
  # print(M1_set.shape)
  M2_set = M2_set.astype('float32')
  M2_set -= np.mean(M2_set)
  M2_set /= np.max(M2_set)
  # print(M2_set.shape)
  M3_set = M3_set.astype('float32')
  M3_set -= np.mean(M3_set)
  M3_set /= np.max(M3_set)
  # print(M3_set.shape)
  M4_set = M4_set.astype('float32')
  M4_set -= np.mean(M4_set)
  M4_set /= np.max(M4_set)
  # print(M4_set.shape)
  M5_set = M5_set.astype('float32')
  M5_set -= np.mean(M5_set)
  M5_set /= np.max(M5_set)
  # print(M5_set.shape)

  M1_set = M1_set.astype('float32')
  M1_set -= np.mean(M1_set)
  M1_set /= np.max(M1_set)
  # print(M1_set.shape)
  M2_set = M2_set.astype('float32')
  M2_set -= np.mean(M2_set)
  M2_set /= np.max(M2_set)
  # print(M2_set.shape)
  M3_set = M3_set.astype('float32')
  M3_set -= np.mean(M3_set)
  M3_set /= np.max(M3_set)
  # print(M3_set.shape)
  M4_set = M4_set.astype('float32')
  M4_set -= np.mean(M4_set)
  M4_set /= np.max(M4_set)
  # print(M4_set.shape)
  M5_set = M5_set.astype('float32')
  M5_set -= np.mean(M5_set)
  M5_set /= np.max(M5_set)
  # print(M5_set.shape)

  M1_set = M1_set.reshape((M1_set.shape[0],1,M1_set.shape[1],M1_set.shape[2],M1_set.shape[3]))
  # print(M1_set.shape)
  M2_set = M2_set.reshape((M2_set.shape[0],1,M2_set.shape[1],M2_set.shape[2],M2_set.shape[3]))
  # print(M2_set.shape)
  M3_set = M3_set.reshape((M3_set.shape[0],1,M3_set.shape[1],M3_set.shape[2],M3_set.shape[3]))
  # print(M3_set.shape)
  M4_set = M4_set.reshape((M4_set.shape[0],1,M4_set.shape[1],M4_set.shape[2],M4_set.shape[3]))
  # print(M4_set.shape)
  M5_set = M5_set.reshape((M5_set.shape[0],1,M5_set.shape[1],M5_set.shape[2],M5_set.shape[3]))
  # print(M5_set.shape)

  return M1_set,M2_set,M3_set,M4_set,M5_set

## Dataset preparation

In [ ]:
dirname_ufc = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/UFC_Data_unzip/A/'
Train_ufc,Test_ufc,Train_ufc_y,Test_ufc_y = apply_H1(dirname_ufc,0,False)

ApplyEyeMakeup loading...
ApplyEyeMakeup loaded
Basketball loading...
Basketball loaded
Diving loading...
Diving loaded
HorseRiding loading...
HorseRiding loaded
PizzaTossing loading...
PizzaTossing loaded
Rowing loading...
Rowing loaded


In [ ]:
M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc = provide_channels(Train_ufc)
# M1_ufct,M2_ufct,M3_ufct,M4_ufct,M5_ufct = provide_channels(Test_ufc)
Train_ufc_y = np.array(Train_ufc_y)
Train_ufc_y = np_utils.to_categorical(Train_ufc_y, 6)
# Test_ufc_y = np.array(Test_ufc_y)
# Test_ufc_y = np_utils.to_categorical(Test_ufc_y, 6)
print("Final channels for UFC train dataset\n",M1_ufc.shape,M2_ufc.shape,M3_ufc.shape,M4_ufc.shape,M5_ufc.shape)
# print("Final channels for UFC test dataset\n",M1_ufct.shape,M2_ufct.shape,M3_ufct.shape,M4_ufct.shape,M5_ufct.shape)
print(Train_ufc_y.shape)
# print(Test_ufc_y.shape)

Final channels for UFC train dataset
 (843, 1, 9, 80, 60) (843, 1, 9, 80, 60) (843, 1, 9, 80, 60) (843, 1, 8, 80, 60) (843, 1, 8, 80, 60)
(843, 6)


In [ ]:
dirname_kth = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/KTH_Data_unzip/'
Train_kth,Test_kth,Train_kth_y,Test_kth_y = apply_H1(dirname_kth,0)

boxing loading...
boxing loaded
handclapping loading...
handclapping loaded
handwaving loading...
handwaving loaded
jogging loading...
jogging loaded
running loading...
running loaded
walking loading...
walking loaded


In [ ]:
M1_kth,M2_kth,M3_kth,M4_kth,M5_kth = provide_channels(Train_kth)
print("Final channels for UFC dataset\n",M1_kth.shape,M2_kth.shape,M3_kth.shape,M4_kth.shape,M5_kth.shape)
# M1_ktht,M2_ktht,M3_ktht,M4_ktht,M5_ktht = provide_channels(Test_kth)
# print("Final channels for UFC test dataset\n",M1_ktht.shape,M2_ktht.shape,M3_ktht.shape,M4_ktht.shape,M5_ktht.shape)
Train_kth_y = np.array(Train_kth_y)
Train_kth_y = np_utils.to_categorical(Train_kth_y, 6)
# Test_kth_y = np.array(Test_kth_y)
# Test_kth_y = np_utils.to_categorical(Test_kth_y, 6)
print(Train_kth_y.shape)
# print(Test_kth_y.shape)

Final channels for UFC dataset
 (599, 1, 9, 80, 60) (599, 1, 9, 80, 60) (599, 1, 9, 80, 60) (599, 1, 8, 80, 60) (599, 1, 8, 80, 60)
(599, 6)


#3D CNN ARCHITECTURE

## Input channels for all the models

In [ ]:
#H1
channel1_input = keras.Input(shape=(1,9,80,60), name="channel1")  #later use variables instead of hearcoding the pixel size and frame number
channel2_input = keras.Input(shape=(1,9,80,60), name="channel2") 
channel3_input = keras.Input(shape=(1,9,80,60), name="channel3") 
channel4_input = keras.Input(shape=(1,8,80,60), name="channel4") 
channel5_input = keras.Input(shape=(1,8,80,60), name="channel5")  
print(channel1_input.shape,channel2_input.shape,channel3_input.shape,channel4_input.shape,channel5_input.shape)
# channel3_input = keras.Input(shape=(num_tags,), name="tags")  # Binary vectors of size `num_tags`

(None, 1, 9, 80, 60) (None, 1, 9, 80, 60) (None, 1, 9, 80, 60) (None, 1, 8, 80, 60) (None, 1, 8, 80, 60)


## Regularisation

### SIFT feature extraction
1. from raw images gray 
2. Motion edge history images

In [ ]:
def SIFT_desc(dirname,MEHI_raw=True,KTH=True,num_frames=9):
  if (KTH):
    actions = ['boxing','handclapping','handwaving','jogging','running','walking']
  else:
    actions = ["ApplyEyeMakeup","Basketball","Diving","HorseRiding","PizzaTossing","Rowing"]
  sift_dataset = []
  for action in actions:
    raw_input_dir = os.listdir(dirname+action)
    print(action,'loading...')
    for file_name in raw_input_dir:
      file_id = dirname + action + '/' + file_name
      capture = cv2.VideoCapture(file_id)
      fps = capture.get(cv2.CAP_PROP_FPS)
      video_features = None
      if (not MEHI_raw):
        success, frame = capture.read()
        frame = cv2.resize(frame,(80,60),interpolation=cv2.INTER_AREA)
        gray1 =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      for i in range(num_frames):
        # print(i)
        if (MEHI_raw): #Raw images
          success, frame = capture.read()
          frame = cv2.resize(frame,(80,60),interpolation=cv2.INTER_AREA)
          gray =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          DSIFT_STEP_SIZE = 6
          sift = cv2.xfeatures2d.SIFT_create()
          disft_step_size = DSIFT_STEP_SIZE
          keypoints = [cv2.KeyPoint(x, y, disft_step_size)
                  for y in range(0, gray.shape[0], disft_step_size)
                      for x in range(0, gray.shape[1], disft_step_size)]
          descriptors = sift.compute(gray, keypoints)[1]
          try:
            video_features = np.concatenate((video_features,np.array(descriptors)),axis=0)
          except:
            video_features = np.array(descriptors)
        else:
          success2, frame2 = capture.read()
          frame2 = cv2.resize(frame2,(80,60),interpolation=cv2.INTER_AREA)
          gray2 =  cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
          D_t_xy = gray1 - gray2
          MEHI = cv2.Canny(D_t_xy,100,200)
          DSIFT_STEP_SIZE = 6
          sift = cv2.xfeatures2d.SIFT_create()
          disft_step_size = DSIFT_STEP_SIZE
          keypoints = [cv2.KeyPoint(x, y, disft_step_size)
                  for y in range(0, MEHI.shape[0], disft_step_size)
                      for x in range(0, MEHI.shape[1], disft_step_size)]
          descriptors = sift.compute(MEHI, keypoints)[1]
          try:
            video_features = np.concatenate((video_features,np.array(descriptors)),axis=0)
          except:
            video_features = np.array(descriptors)
          gray1 = gray2

      sift_dataset.append(video_features)
    print(action,'loaded')
  # Training_set = np.array(Training_set)
  return sift_dataset

In [ ]:
sift_whole = SIFT_desc(dirname_ufc,True,False,9)
sift_whole_np = np.array(sift_whole)
print(sift_whole_np.shape)
sift_whole_MEHI = SIFT_desc(dirname_ufc,False,False,9)
sift_whole_MEHI_np = np.array(sift_whole_MEHI)
print(sift_whole_MEHI_np.shape)

ApplyEyeMakeup loading...
ApplyEyeMakeup loaded
Basketball loading...
Basketball loaded
Diving loading...
Diving loaded
HorseRiding loading...
HorseRiding loaded
PizzaTossing loading...
PizzaTossing loaded
Rowing loading...
Rowing loaded
(843, 1260, 128)
ApplyEyeMakeup loading...
ApplyEyeMakeup loaded
Basketball loading...
Basketball loaded
Diving loading...
Diving loaded
HorseRiding loading...
HorseRiding loaded
PizzaTossing loading...
PizzaTossing loaded
Rowing loading...
Rowing loaded
(843, 1260, 128)


### Soft-feature Quantisation, GMM

Softly quantized (each local feature can be assigned to multiple codebook words) using a 512-word codebook. The method is basically Multiple clustering where each data point can be assigned to multiple clusters with each point having the probability of belonging to each cluster. Such a model is GMM.

Not using the whole dataset for GMM model because of RAM constraints. Instead sampling random features with the assumption that the randomly chosen arguments would also give a fair representation of the complete distribution.

In [ ]:
def cluster_features(img_descs,cluster_model,n_clusters=512):
    # n_clusters = cluster_model.n_clusters
    des_stack_train=[desc for img in img_descs for desc in img]
    all_train_descriptors = np.array(des_stack_train)

    if all_train_descriptors.shape[1] != 128:
        raise ValueError('Expected SIFT descriptors to have 128 features, got', all_train_descriptors.shape[1])

    print ('%i Descriptors before clustering' % all_train_descriptors.shape[0])

    randomly_sampled = all_train_descriptors[np.random.choice(all_train_descriptors.shape[0], size=30000, replace=False), :]
    # randomly_sampled = np.random.choice(sift_whole_np, size=10000, replace=False)
    #If data does not fit in memory you can find a way to randomly sample when you read it
    print(randomly_sampled.shape)

    print ('Using clustering model %s...' % repr(cluster_model))
    print ('Clustering on training set to get codebook of %i words' % n_clusters)

    # train kmeans or other cluster model on those descriptors selected above
    cluster_model.fit(randomly_sampled)
    print ('done clustering.')

    return cluster_model

In [ ]:
GMM_model = GaussianMixture(n_components=512)
cluster_model_raw = cluster_features(sift_whole_np,GMM_model)
GMM_model_MEHI = GaussianMixture(n_components=512)
cluster_model_MEHI = cluster_features(sift_whole_MEHI_np,GMM_model_MEHI)

1062180 Descriptors before clustering
(30000, 128)
Using clustering model GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=512, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)...
Clustering on training set to get codebook of 512 words
done clustering.
1062180 Descriptors before clustering
(30000, 128)
Using clustering model GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=512, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)...
Clustering on training set to get codebook of 512 words
done clustering.


### Encoding the features

In [ ]:
def extract_denseSIFT(gray):
    DSIFT_STEP_SIZE = 6
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, gray.shape[0], disft_step_size)
                for x in range(0, gray.shape[1], disft_step_size)]

    descriptors = sift.compute(gray, keypoints)[1]
    
    #keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

### Spatial Pyramid Matching

In [ ]:
def getImageFeaturesSPM(w_step,h_step, file_id, cluster_model, K,raw_MEHI=True,W=80, H=60,num_frames=9):  
    h = []
    x = 0
    y = 0
    for i in range(w_step):
      for j in range(h_step):
        capture = cv2.VideoCapture(file_id)
        fps = capture.get(cv2.CAP_PROP_FPS)
        desc_ = None
        if (not raw_MEHI):
          success, frame = capture.read()
          frame = cv2.resize(frame,(80,60),interpolation=cv2.INTER_AREA)
          img1 =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        for k in range(num_frames):
            # print(k)
            if (raw_MEHI):
              success, frame = capture.read()
              frame = cv2.resize(frame,(80,60),interpolation=cv2.INTER_AREA)
              img =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              desc = extract_denseSIFT(img[y:y+w_step, x:x+h_step])
              try: 
                desc_ = np.concatenate((desc_,desc),axis=0)
              except:
                desc_ = desc
            else:
              success2, frame2 = capture.read()
              frame2 = cv2.resize(frame2,(80,60),interpolation=cv2.INTER_AREA)
              img2 =  cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
              D = img1 - img2
              H = cv2.Canny(D,100,200)
              desc = extract_denseSIFT(H[y:y+w_step, x:x+h_step])
              try: 
                desc_ = np.concatenate((desc_,desc),axis=0)
              except:
                desc_ = desc
        capture.release()
        predict = cluster_model.predict(desc_)
        histo = np.bincount(predict, minlength=K).reshape(1,-1).ravel()
        h.append(histo)
        x = x + w_step
      y = y + h_step
    
    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    hist = hist - np.mean(hist)
    hist /= dev
    return hist

In [ ]:
def aux_units_gray_MEHI(dirname,raw_MEHI=True,KTH=True,num_frames=9):
  if (KTH):
    actions = ['boxing','handclapping','handwaving','jogging','running','walking']
  else:
    actions = ["ApplyEyeMakeup","Basketball","Diving","HorseRiding","PizzaTossing","Rowing"]
  sift_dataset = []
  for action in actions:
    raw_input_dir = os.listdir(dirname+action)
    # print(action,'loading...')
    for file_name in raw_input_dir:
      file_id = dirname + action + '/' + file_name
      if (raw_MEHI):
        res1_gray = getImageFeaturesSPM(2,2,file_id,cluster_model_raw,512,True)
        res2_gray = getImageFeaturesSPM(3,4,file_id,cluster_model_raw,512,True)
        SPM_Gray = np.concatenate((res1_gray,res2_gray))
        sift_dataset.append(SPM_Gray)
      else:
        res1_MEHI = getImageFeaturesSPM(2,2,file_id,cluster_model_MEHI,512,False)
        res2_MEHI = getImageFeaturesSPM(3,4,file_id,cluster_model_MEHI,512,False)
        SPM_MEHI = np.concatenate((res1_MEHI,res2_MEHI)) 
        sift_dataset.append(SPM_MEHI)
    # print(action,'loaded')
  # Training_set = np.array(Training_set)
  return sift_dataset

In [ ]:
aux_output_units_g = aux_units_gray_MEHI(dirname_ufc,True,False)
aux_output_units_g_np = np.array(aux_output_units_g)
print(aux_output_units_g_np.shape)
aux_output_units_m = aux_units_gray_MEHI(dirname_ufc,False,False)
aux_output_units_m_np = np.array(aux_output_units_m)
print(aux_output_units_m_np.shape)

(843, 8192)
(843, 8192)


In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

In [ ]:
print(aux_output_units_m_np.shape)
sc = StandardScaler() 
pca_SPM_MEHI = sc.fit_transform(aux_output_units_m_np)
# print(pca_SPM_MEHI.shape)
pca = PCA(n_components = 150) 
pca_SPM_MEHI = pca.fit_transform(pca_SPM_MEHI)
print(pca_SPM_MEHI.shape) 
#for raw gray images
print(aux_output_units_g_np.shape)
sc = StandardScaler() 
pca_SPM_GRAY = sc.fit_transform(aux_output_units_g_np)
# print(pca_SPM_GRAY.shape)
pca = PCA(n_components = 150) 
pca_SPM_GRAY = pca.fit_transform(pca_SPM_GRAY)
print(pca_SPM_GRAY.shape) 
pca_SPM_cube_ufc = np.concatenate((pca_SPM_GRAY,pca_SPM_MEHI),axis=1)
print(pca_SPM_cube_ufc.shape)

(843, 8192)
(843, 150)
(843, 8192)
(843, 150)
(843, 300)


In [ ]:
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/pca_spm_cube.pickle'
with open(filename, 'wb') as f:
    pickle.dump(pca_SPM_cube_ufc, f)

In [ ]:
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/pca_spm_cube.pickle'
with open(filename, 'rb') as f:
    pca_SPM_cube_ufc_reloaded = pickle.load(f)

In [ ]:
pca_SPM_cube_ufc_reloaded == pca_SPM_cube_ufc

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

## Generic architectures
* for writing and compiling the function
* Defining a function that can create all architectures, using keras inbuilt libraries

In [ ]:
def CNN_3D(channel1_inputs,channel2_inputs,channel3_inputs,channel4_inputs,channel5_inputs,architecture='s332',activation_function='relu',combination=5,optimizer_c='RMSProp'):

  if (architecture == 's332') or (architecture == 'rs332'):
    print(architecture)
    kernel_size1 = (3,9,7)
    kernel_size2 = (3,7,7)
    final_ = 138
    if (architecture == 's332'):
      name_128 = "shape128_s332"
      name_output = "output_s332"
    else:
      name_128 = "shape128_rs332"
      name_output = "output_rs332"

  if (architecture == 'm332'):
    print(architecture)
    kernel_size1 = (3,9,7)
    kernel_size2 = (3,7,7)
    final_ = 162
    name_128 = "shape128_m332"
    name_output = "output_m332"
  
  if (architecture == 'm322'):
    print(architecture)
    kernel_size1 = (3,9,7)
    kernel_size2 = (1,7,7)
    final_ = 198
    name_128 = "shape128_m322"
    name_output = "output_m322"

  if (architecture == 'm222'):
    print(architecture)
    kernel_size1 = (1,9,7)
    kernel_size2 = (1,7,7)
    final_ = 258
    name_128 = "shape128_m222"
    name_output = "output_m222"


  if (architecture == "ensemble"):
    if (combination==1):
      #rs332
      modelr332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"rs332")
      return modelr332
    elif (combination==2):
      #rs332 + s332
      model_s332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input)
      model_r332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"rs332")
      ensemble_layer = keras.layers.Average(name="ens_res")([model_s332.outputs[0],model_r332.outputs[0]])
    elif (combination==3):
      #rs332 + s332 + m222
      model_s332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input)
      model_r332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"rs332")
      model_m222 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m222")
      ensemble_layer = keras.layers.Average(name="ens_res")([model_s332.outputs[0],model_r332.outputs[0],model_m222.outputs[0]])
    elif (combination==4):
      #rs332 + s332 + m222 + m322
      model_s332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input)
      model_r332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"rs332")
      model_m322 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m322")
      model_m222 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m222")
      ensemble_layer = keras.layers.Average(name="ens_res")([model_s332.outputs[0],model_r332.outputs[0],model_m322.outputs[0],model_m222.outputs[0]])
    elif (combination==5):
      #rs332 + s332 + m222 + m322 + m332
      model_s332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input)
      model_r332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"rs332")
      model_m332 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m332")
      model_m322 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m322")
      model_m222 = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m222")
      ensemble_layer = keras.layers.Average(name="ens_res")([model_s332.outputs[0],model_r332.outputs[0],model_m332.outputs[0],model_m322.outputs[0],model_m222.outputs[0]])
    ensemble_model = keras.Model(
        inputs=[channel1_input,channel2_input,channel3_input,channel4_input,channel5_input],
        outputs = [ensemble_layer]
    )
    ensemble_model.compile(loss='categorical_crossentropy', metrics=['mse', 'accuracy',keras.metrics.Precision()])
    return ensemble_model

  #C2
  channel1_features = keras.layers.Conv3D(2,kernel_size=kernel_size1,data_format='channels_first',activation=activation_function)(channel1_input)
  channel2_features = keras.layers.Conv3D(2,kernel_size=kernel_size1,data_format='channels_first',activation=activation_function)(channel2_input)
  channel3_features = keras.layers.Conv3D(2,kernel_size=kernel_size1,data_format='channels_first',activation=activation_function)(channel3_input)
  channel4_features = keras.layers.Conv3D(2,kernel_size=kernel_size1,data_format='channels_first',activation=activation_function)(channel4_input)
  channel5_features = keras.layers.Conv3D(2,kernel_size=kernel_size1,data_format='channels_first',activation=activation_function)(channel5_input)
  print(channel1_features.shape,channel2_features.shape,channel3_features.shape,channel4_features.shape,channel5_features.shape)

  #S3
  channel1_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel1_features)

  if (architecture == 's332') or (architecture == 'rs332'):
    #normal
    channel2_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel2_features)
    channel3_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel3_features)
    channel4_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel4_features)
    channel5_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel5_features)
    print(channel1_features.shape,channel2_features.shape,channel3_features.shape,channel4_features.shape,channel5_features.shape)
  elif ((architecture == "m332") or (architecture == "m322") or (architecture == "m222")):
    #S3 (First mixing the channels of same type and subsampling) 
    channel2_features = keras.layers.concatenate(inputs=[channel2_features, channel3_features],axis=2)
    channel3_features = keras.layers.concatenate(inputs=[channel4_features,channel5_features],axis=2)

    channel2_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel2_features)
    channel3_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel3_features)
    print(channel1_features.shape,channel2_features.shape,channel3_features.shape)
  

  #C4
  channel1_features = tf.keras.layers.Conv3D(6,kernel_size=kernel_size2,data_format='channels_first',activation=activation_function,groups=2)(channel1_features)
  channel2_features = tf.keras.layers.Conv3D(6,kernel_size=kernel_size2,data_format='channels_first',activation=activation_function,groups=2)(channel2_features)
  channel3_features = tf.keras.layers.Conv3D(6,kernel_size=kernel_size2,data_format='channels_first',activation=activation_function,groups=2)(channel3_features)

  if (architecture == 's332') or (architecture == 'rs332'):
    channel4_features = tf.keras.layers.Conv3D(6,kernel_size=kernel_size2,data_format='channels_first',activation=activation_function,groups=2)(channel4_features)
    channel5_features = tf.keras.layers.Conv3D(6,kernel_size=kernel_size2,data_format='channels_first',activation=activation_function,groups=2)(channel5_features)
    print(channel1_features.shape,channel2_features.shape,channel3_features.shape,channel4_features.shape,channel5_features.shape)
  elif ((architecture == "m332") or (architecture == "m322") or (architecture == "m222")):
    print(channel1_features.shape,channel2_features.shape,channel3_features.shape)

  #S5
  if (architecture == 's332') or (architecture == 'rs332'):
    channel1_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel1_features)
    channel2_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel2_features)
    channel3_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel3_features)
    channel4_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel4_features)
    channel5_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel5_features)
    print(channel1_features.shape,channel2_features.shape,channel3_features.shape,channel4_features.shape,channel5_features.shape)
  elif ((architecture == "m332") or (architecture == "m322") or (architecture == "m222")):
    channel1_features = keras.layers.concatenate(inputs=[channel1_features,channel2_features, channel3_features],axis=2)
    channel1_features = keras.layers.AveragePooling3D(pool_size=(1,3,3),data_format="channels_first")(channel1_features)
    print(channel1_features.shape)

  #C6
  channel1_features = keras.layers.Conv3D(filters=6,kernel_size=(1,6,4),data_format='channels_first',activation='relu')(channel1_features)
  if (architecture == 's332') or (architecture == 'rs332'):
    channel2_features = keras.layers.Conv3D(filters=6,kernel_size=(1,6,4),data_format='channels_first',activation=activation_function)(channel2_features)
    channel3_features = keras.layers.Conv3D(filters=6,kernel_size=(1,6,4),data_format='channels_first',activation=activation_function)(channel3_features)
    channel4_features = keras.layers.Conv3D(filters=6,kernel_size=(1,6,4),data_format='channels_first',activation=activation_function)(channel4_features)
    channel5_features = keras.layers.Conv3D(filters=6,kernel_size=(1,6,4),data_format='channels_first',activation=activation_function)(channel5_features)
    print(channel1_features.shape,channel2_features.shape,channel3_features.shape,channel4_features.shape,channel5_features.shape)
  elif ((architecture == "m332") or (architecture == "m322") or (architecture == "m222")):
    print(channel1_features.shape)

  if (architecture == 's332') or (architecture == 'rs332'):
    features = keras.layers.concatenate(inputs=[channel1_features, channel2_features, channel3_features,channel4_features,channel5_features],axis=2)
    print(features.shape)
  elif ((architecture == "m332") or (architecture == "m322") or (architecture == "m222")):
    features = channel1_features

  features = tf.keras.layers.Reshape(target_shape=(final_,),)(features)
  print(features.shape)

  features = tf.keras.layers.Dense(128,name=name_128)(features)
  print(features.shape)

  # if (architecture == "rs322"):
  #   pca_features = tf.keras.layers.Dense(300,activation='relu', name='output_spm_rs332')(features)
  #   print(pca_features.shape)

  final_features = tf.keras.layers.Dense(6,activation='softmax',name=name_output)(features)
  print(final_features.shape)

  # softmax_layer = Dense(1,activation='softmax')(final_features)
  # print(softmax_layer.shape)

  if (architecture == 'rs332'):
    print("correct")
    pca_features = tf.keras.layers.Dense(300,activation='relu', name='output_spm_rs332')(features)
    print(pca_features.shape)
    model = keras.Model(
        inputs=[channel1_input,channel2_input,channel3_input,channel4_input,channel5_input],
        outputs = [final_features,pca_features])
    model.compile(
        optimizer='RMSprop',
        loss={"output_rs332": keras.losses.categorical_crossentropy,"output_spm_rs332": keras.losses.mean_squared_error},
        loss_weights=[1.0, 0.005],
        metrics={"output_rs332":["mse","accuracy",keras.metrics.Precision()],"output_spm_rs332":['mse', 'accuracy']},)
    return model

  model = keras.Model(
      inputs=[channel1_input,channel2_input,channel3_input,channel4_input,channel5_input],
      outputs = [final_features]
  )

  model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['mse', 'accuracy',keras.metrics.Precision()])
  return model

## Averaging all five models

In [ ]:
# model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input)
# model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m332")
# model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m322")
# model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m222")
model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"s332","tanh",None,"RMSProp")
model.summary()
del model
keras.backend.clear_session()

s332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
channel1 (InputLayer)           [(None, 1, 9, 80, 60 0                                            
__________________________________________________________________________________________________
channel2 (InputLay

# EXPERIMENTS

In [ ]:
def kfold_validation(model_type,M1,M2,M3,M4,M5,Y,spm,batch_size=32,no_epochs=30,verbosity=1,combination=5,activation='relu',optimizer='RMSProp'): 
  num_folds = 5
  loss_per_fold = []
  acc_per_fold = []
  prec_per_fold = []
  mse_per_fold = []
  # Merge inputs and targets
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(M1, Y):

    output_name = "output_" + model_type

    if (model_type=="s332"):
      model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"s332",activation,None,optimizer)
      print("New s332 model initialised")
    elif (model_type=="m332"):
      model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m332",activation,None,optimizer)
      print("New m332 model initialised")
    elif (model_type=="m322"):
      model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m322",activation,None,optimizer)
      print("New m322 model initialised")
    elif (model_type=="m222"):
      model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"m222",activation,None,optimizer)
      print("New m222 model initialised")
    elif (model_type=="rs332"):
      model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"rs332",activation,None,optimizer)
      print("New rs332 model initialised")
    elif (model_type=="ensemble"):
      model = CNN_3D(channel1_input,channel2_input,channel3_input,channel4_input,channel5_input,"ensemble",activation,5,optimizer)
      print("New ensemble model initialised")

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    if (model_type == "rs332"):
      history = model.fit(
                  {"channel1": M1[train], "channel2": M2[train], "channel3": M3[train],"channel4":M4[train],"channel5":M5[train]},
                  {output_name:Y[train],"output_spm_rs332":spm[train]},
                  batch_size=batch_size,
                  epochs=no_epochs,
                  verbose=verbosity)
      # Generate generalization metrics
      scores = model.evaluate({"channel1":M1[test],"channel2":M2[test],"channel3":M3[test],"channel4":M4[test],"channel5":M5[test]}, 
                              {output_name:Y[test],"output_spm_rs332":spm[test]}, 
                              verbose=0)
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]};{model.metrics_names[2]} of {scores[2]};{model.metrics_names[3]} of {scores[3]}')
      acc_per_fold.append(scores[4])
      loss_per_fold.append(scores[1])
      prec_per_fold.append(scores[5])
      mse_per_fold.append(scores[3])
    else:
      if (model_type=="ensemble"):
        output_name = "ens_res"
      history = model.fit(
                  {"channel1": M1[train], "channel2": M2[train], "channel3": M3[train],"channel4":M4[train],"channel5":M5[train]},
                  {output_name:Y[train]},
                  batch_size=batch_size,
                  epochs=no_epochs,
                  verbose=verbosity)
    
      # Generate generalization metrics
      scores = model.evaluate({"channel1":M1[test],"channel2":M2[test],"channel3":M3[test],"channel4":M4[test],"channel5":M5[test]}, 
                              {output_name:Y[test]}, 
                              verbose=0)
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]};{model.metrics_names[2]} of {scores[2]};{model.metrics_names[3]} of {scores[3]}')
      acc_per_fold.append(scores[2])
      loss_per_fold.append(scores[0])
      prec_per_fold.append(scores[3])
      mse_per_fold.append(scores[1])

    del model
    keras.backend.clear_session()


    # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
  print('------------------------------------------------------------------------')
  print('Score per fold')
  for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - Precision: {prec_per_fold[i]}%')
  print('------------------------------------------------------------------------')
  print('Average scores for all folds:')
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  print(f'> Precision: {np.mean(prec_per_fold)}')
  print('------------------------------------------------------------------------')
  return np.array(loss_per_fold),np.array(mse_per_fold),np.array(acc_per_fold),np.array(prec_per_fold)


## ReLu and RMSProp

### Epochs - 30

#### Model s332

In [ ]:
results_diff = {}
loss,mse,acc,prec = kfold_validation("s332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30)
results_diff["s332"] = np.array((loss,mse,acc,prec))
#the error has been resolved
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/s332_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff["s332"], f)

s332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
New s332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 6s 260ms/step - loss: 1.5721 - mse: 0.1245 - accuracy: 0.3501 - precision: 0.7538
Epoch 2/30
22/22 [==============================] - 5s 248ms/step - loss: 1.1720 - mse: 0.0966 - accuracy: 0.5593 - precision: 0.7610
Epoch 3/30
22/22 [==============================] - 5s 249ms/step - loss: 0.9811 - mse: 0.0819 - accuracy: 0.6350 - precision: 0.7663
Epoch 4/30
22/22 [==============================] - 6s 251ms/step - loss: 0.8180 - mse: 0.0692 - accuracy: 0.7062 - precision: 0.8013
Epoch 5/30
22/22 [==============================] - 5s 248ms/step - loss: 0.7351 - mse: 0.0616 - accuracy: 0.7226 - precision: 0.8275
Epoch 6/30
22/22 [==============================] - 5s 249ms/step - loss: 0.6725 - mse: 0.0549 - accuracy: 0.7611 - precision: 0.8521
Epoch 7/30
22/22 [==============================] - 5s 248ms/step - loss: 0.5774 - mse: 0.0483 - accuracy: 0.8056 - precision: 0.8604
Epoch 8/30
22/22 [==============================] - 5s 249ms/s

ValueError: ignored

#### Model m332

In [ ]:
loss,mse,acc,prec = kfold_validation("m332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30)
results_diff["m332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m332_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff["m332"], f)

m332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 14, 24, 18) (None, 2, 12, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 12, 18, 12) (None, 6, 10, 18, 12)
(None, 6, 27, 6, 4)
(None, 6, 27, 1, 1)
(None, 162)
(None, 128)
(None, 6)
New m332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 5s 249ms/step - loss: 1.5291 - mse: 0.1214 - accuracy: 0.4065 - precision: 0.6875
Epoch 2/30
22/22 [==============================] - 5s 246ms/step - loss: 1.0734 - mse: 0.0879 - accuracy: 0.6128 - precision: 0.7647
Epoch 3/30
22/22 [==============================] - 5s 247ms/step - loss: 0.8945 - mse: 0.0756 - accuracy: 0.6691 - precision: 0.7630
Epoch 4/30
22/22 [==============================] - 5s 246ms/step - loss: 0.7990 - mse: 0.0688 - accuracy: 0.6840 - precision: 0.7881
Epoch 5/30
22/22 [==============================] - 5s 245ms/step - loss: 0.7132 - mse: 0.0620 - accuracy: 0.7196 - precision: 0.7941
Epoch 6/30
22/22 [==============================] - 5s 245ms/step - loss: 0.6364 - mse: 0.0546 - accuracy: 0.7626 - precision: 0.8407
Epoch 7/30
22/22 [==============================] - 5s 246ms/step - loss: 0.6126 - mse: 0.0536 - accuracy: 0.7671 - precision: 0.8187
Epoch 8/30
22/22 [==============================] - 5s 247ms/s

In [ ]:
keras.backend.clear_session()

#### Model m322

In [ ]:
loss,mse,acc,prec = kfold_validation("m322",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30)
results_diff["m322"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m322_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff["m322"], f)

m322
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 14, 24, 18) (None, 2, 12, 24, 18)
(None, 6, 7, 18, 12) (None, 6, 14, 18, 12) (None, 6, 12, 18, 12)
(None, 6, 33, 6, 4)
(None, 6, 33, 1, 1)
(None, 198)
(None, 128)
(None, 6)
New m322 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 5s 245ms/step - loss: 1.6948 - mse: 0.1332 - accuracy: 0.2493 - precision: 0.6000
Epoch 2/30
22/22 [==============================] - 5s 242ms/step - loss: 1.3275 - mse: 0.1105 - accuracy: 0.4540 - precision: 0.6726
Epoch 3/30
22/22 [==============================] - 5s 240ms/step - loss: 1.1646 - mse: 0.0980 - accuracy: 0.5564 - precision: 0.6677
Epoch 4/30
22/22 [==============================] - 5s 239ms/step - loss: 0.9849 - mse: 0.0855 - accuracy: 0.6157 - precision: 0.7188
Epoch 5/30
22/22 [==============================] - 5s 240ms/step - loss: 0.8990 - mse: 0.0780 - accuracy: 0.6766 - precision: 0.7406
Epoch 6/30
22/22 [==============================] - 5s 241ms/step - loss: 0.7710 - mse: 0.0673 - accuracy: 0.7181 - precision: 0.7769
Epoch 7/30
22/22 [==============================] - 5s 240ms/step - loss: 0.7679 - mse: 0.0668 - accuracy: 0.7122 - precision: 0.7729
Epoch 8/30
22/22 [==============================] - 5s 239ms/s

#### Model m222 

In [ ]:
loss,mse,acc,prec = kfold_validation("m222",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30)
results_diff["m222"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m222_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff["m222"], f)

m222
(None, 2, 9, 72, 54) (None, 2, 9, 72, 54) (None, 2, 9, 72, 54) (None, 2, 8, 72, 54) (None, 2, 8, 72, 54)
(None, 2, 9, 24, 18) (None, 2, 18, 24, 18) (None, 2, 16, 24, 18)
(None, 6, 9, 18, 12) (None, 6, 18, 18, 12) (None, 6, 16, 18, 12)
(None, 6, 43, 6, 4)
(None, 6, 43, 1, 1)
(None, 258)
(None, 128)
(None, 6)
New m222 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 4s 184ms/step - loss: 1.6736 - mse: 0.1316 - accuracy: 0.3012 - precision: 0.8462
Epoch 2/30
22/22 [==============================] - 4s 173ms/step - loss: 1.2653 - mse: 0.1052 - accuracy: 0.5059 - precision: 0.6900
Epoch 3/30
22/22 [==============================] - 4s 173ms/step - loss: 1.0053 - mse: 0.0854 - accuracy: 0.6246 - precision: 0.7620
Epoch 4/30
22/22 [==============================] - 4s 173ms/step - loss: 0.8385 - mse: 0.0719 - accuracy: 0.6751 - precision: 0.8128
Epoch 5/30
22/22 [==============================] - 4s 171ms/step - loss: 0.7851 - mse: 0.0672 - accuracy: 0.7062 - precision: 0.7922
Epoch 6/30
22/22 [==============================] - 4s 172ms/step - loss: 0.6982 - mse: 0.0595 - accuracy: 0.7433 - precision: 0.8286
Epoch 7/30
22/22 [==============================] - 4s 172ms/step - loss: 0.6655 - mse: 0.0567 - accuracy: 0.7493 - precision: 0.8370
Epoch 8/30
22/22 [==============================] - 4s 173ms/s

#### Model rs332

In [ ]:
loss,mse,acc,prec = kfold_validation("rs332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30)
results_diff["rs332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/rs332_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff["rs332"], f)

rs332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
correct
(None, 300)
New rs332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 6s 261ms/step - loss: 1.8873 - output_rs332_loss: 1.6810 - output_spm_rs332_loss: 41.2598 - output_rs332_mse: 0.1311 - output_rs332_accuracy: 0.3012 - output_rs332_precision: 0.7297 - output_spm_rs332_mse: 41.2598 - output_spm_rs332_accuracy: 0.0104
Epoch 2/30
22/22 [==============================] - 6s 256ms/step - loss: 1.5812 - output_rs332_loss: 1.3751 - output_spm_rs332_loss: 41.2178 - output_rs332_mse: 0.1108 - output_rs332_accuracy: 0.4733 - output_rs332_precision: 0.7035 - output_spm_rs332_mse: 41.2178 - output_spm_rs332_accuracy: 0.1543
Epoch 3/30
22/22 [==============================] - 6s 256ms/step - loss: 1.3449 - output_rs332_loss: 1.1390 - output_spm_rs332_loss: 41.1878 - output_rs332_mse: 0.0942 - output_rs332_accuracy: 0.5579 - output_rs332_precision: 0.7311 - output_spm_rs332_mse: 41.1878 - output_spm_rs332_accuracy: 0.2478
Epoch 4/30
22/22 [==============================] - 6s 257ms/step - loss: 1.2250 - output_rs33

#### Average of all five

In [ ]:
loss,mse,acc,prec = kfold_validation("ensemble",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30,1,5)
results_diff["ensemble5"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/ensemble5_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff["ensemble5"], f)

s332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
rs332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 23s 1s/step - loss: 1.5735 - mse: 0.1252 - accuracy: 0.3680 - precision_5: 0.9167
Epoch 2/30
22/22 [==============================] - 23s 1s/step - loss: 1.1680 - mse: 0.0978 - accuracy: 0.5920 - precision_5: 0.8529
Epoch 3/30
22/22 [==============================] - 23s 1s/step - loss: 0.9895 - mse: 0.0842 - accuracy: 0.6350 - precision_5: 0.8645
Epoch 4/30
22/22 [==============================] - 23s 1s/step - loss: 0.8379 - mse: 0.0726 - accuracy: 0.7122 - precision_5: 0.8469
Epoch 5/30
22/22 [==============================] - 23s 1s/step - loss: 0.7692 - mse: 0.0667 - accuracy: 0.7285 - precision_5: 0.8548
Epoch 6/30
22/22 [==============================] - 23s 1s/step - loss: 0.6790 - mse: 0.0598 - accuracy: 0.7789 - precision_5: 0.8959
Epoch 7/30
22/22 [==============================] - 23s 1s/step - loss: 0.6027 - mse: 0.0527 - accuracy: 0.8160 - precision_5: 0.9085
Epoch 8/30
22/22 [==============================] - 23s 1s/ste

In [ ]:
print(results_diff)
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/all_tanh_RMSProp_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_diff, f)

{'s332': array([[1.73843312, 2.97069669, 1.9106369 , 2.27589703, 1.62567008],
       [0.06662584, 0.1085136 , 0.08502801, 0.10118589, 0.07481257],
       [0.77514791, 0.63313609, 0.68639052, 0.65476191, 0.72619045],
       [0.77976191, 0.63313609, 0.70303029, 0.65662652, 0.7305389 ]]), 'm332': array([[1.7556864 , 1.61394954, 1.39328837, 1.53206849, 1.45437217],
       [0.09262872, 0.0756221 , 0.08638125, 0.08375923, 0.07239294],
       [0.6745562 , 0.71597636, 0.69230771, 0.69047618, 0.73809522],
       [0.68711656, 0.73456788, 0.68862277, 0.70303029, 0.75      ]]), 'm322': array([[1.5167408 , 1.55874836, 1.00623989, 1.41281676, 1.14461493],
       [0.08541694, 0.07974398, 0.0698203 , 0.08627059, 0.08041979],
       [0.70414203, 0.72189349, 0.72781062, 0.66071427, 0.70238096],
       [0.7037037 , 0.72891569, 0.74233127, 0.66666669, 0.71951222]]), 'm222': array([[1.65126264, 1.31799638, 1.91585577, 1.20069337, 1.52067256],
       [0.08936648, 0.08787273, 0.11723122, 0.07197239, 0.082434

## Tanh and RMSProp

### Model s332

In [ ]:
results_tanh = {}
loss,mse,acc,prec = kfold_validation("s332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"tanh")
results_tanh["s332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/s332_30epochs_tanh.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh["s332"], f)

s332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
New s332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 6s 251ms/step - loss: 1.4226 - mse: 0.1138 - accuracy: 0.4303 - precision: 0.8060
Epoch 2/25
22/22 [==============================] - 5s 249ms/step - loss: 1.0743 - mse: 0.0913 - accuracy: 0.5816 - precision: 0.7249
Epoch 3/25
22/22 [==============================] - 5s 249ms/step - loss: 0.9389 - mse: 0.0799 - accuracy: 0.6454 - precision: 0.7770
Epoch 4/25
22/22 [==============================] - 5s 249ms/step - loss: 0.8602 - mse: 0.0729 - accuracy: 0.6884 - precision: 0.7751
Epoch 5/25
22/22 [==============================] - 6s 250ms/step - loss: 0.8029 - mse: 0.0679 - accuracy: 0.7107 - precision: 0.7914
Epoch 6/25
22/22 [==============================] - 5s 249ms/step - loss: 0.7357 - mse: 0.0616 - accuracy: 0.7537 - precision: 0.8244
Epoch 7/25
22/22 [==============================] - 5s 249ms/step - loss: 0.6851 - mse: 0.0584 - accuracy: 0.7685 - precision: 0.8294
Epoch 8/25
22/22 [==============================] - 5s 249ms/s

### Model m332

In [ ]:
loss,mse,acc,prec = kfold_validation("m332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"tanh")
results_tanh["m332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m332_25epochs_tanh.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh["m332"], f)

m332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 14, 24, 18) (None, 2, 12, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 12, 18, 12) (None, 6, 10, 18, 12)
(None, 6, 27, 6, 4)
(None, 6, 27, 1, 1)
(None, 162)
(None, 128)
(None, 6)
New m332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 5s 246ms/step - loss: 1.4787 - mse: 0.1179 - accuracy: 0.4080 - precision: 0.7404
Epoch 2/25
22/22 [==============================] - 5s 244ms/step - loss: 1.0736 - mse: 0.0900 - accuracy: 0.5623 - precision: 0.7485
Epoch 3/25
22/22 [==============================] - 5s 243ms/step - loss: 0.9293 - mse: 0.0784 - accuracy: 0.6558 - precision: 0.7672
Epoch 4/25
22/22 [==============================] - 5s 245ms/step - loss: 0.8316 - mse: 0.0708 - accuracy: 0.6825 - precision: 0.7978
Epoch 5/25
22/22 [==============================] - 5s 246ms/step - loss: 0.7542 - mse: 0.0639 - accuracy: 0.7226 - precision: 0.8027
Epoch 6/25
22/22 [==============================] - 5s 246ms/step - loss: 0.6814 - mse: 0.0586 - accuracy: 0.7582 - precision: 0.8177
Epoch 7/25
22/22 [==============================] - 5s 245ms/step - loss: 0.6127 - mse: 0.0530 - accuracy: 0.7685 - precision: 0.8499
Epoch 8/25
22/22 [==============================] - 5s 246ms/s

### Model m322

In [ ]:
loss,mse,acc,prec = kfold_validation("m322",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"tanh")
results_tanh["m322"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m322_25epochs_tanh.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh["m322"], f)

m322
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 14, 24, 18) (None, 2, 12, 24, 18)
(None, 6, 7, 18, 12) (None, 6, 14, 18, 12) (None, 6, 12, 18, 12)
(None, 6, 33, 6, 4)
(None, 6, 33, 1, 1)
(None, 198)
(None, 128)
(None, 6)
New m322 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 5s 245ms/step - loss: 1.5428 - mse: 0.1222 - accuracy: 0.3858 - precision: 0.8772
Epoch 2/25
22/22 [==============================] - 5s 241ms/step - loss: 1.0771 - mse: 0.0889 - accuracy: 0.6142 - precision: 0.7729
Epoch 3/25
22/22 [==============================] - 5s 241ms/step - loss: 0.9029 - mse: 0.0751 - accuracy: 0.6869 - precision: 0.7964
Epoch 4/25
22/22 [==============================] - 5s 241ms/step - loss: 0.7936 - mse: 0.0669 - accuracy: 0.7151 - precision: 0.7930
Epoch 5/25
22/22 [==============================] - 5s 242ms/step - loss: 0.7294 - mse: 0.0623 - accuracy: 0.7404 - precision: 0.8304
Epoch 6/25
22/22 [==============================] - 5s 241ms/step - loss: 0.6420 - mse: 0.0544 - accuracy: 0.7774 - precision: 0.8394
Epoch 7/25
22/22 [==============================] - 5s 241ms/step - loss: 0.5921 - mse: 0.0510 - accuracy: 0.7834 - precision: 0.8551
Epoch 8/25
22/22 [==============================] - 5s 241ms/s

### Model m222

In [ ]:
loss,mse,acc,prec = kfold_validation("m222",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"tanh")
results_tanh["m222"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m222_25epochs_tanh.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh["m222"], f)

m222
(None, 2, 9, 72, 54) (None, 2, 9, 72, 54) (None, 2, 9, 72, 54) (None, 2, 8, 72, 54) (None, 2, 8, 72, 54)
(None, 2, 9, 24, 18) (None, 2, 18, 24, 18) (None, 2, 16, 24, 18)
(None, 6, 9, 18, 12) (None, 6, 18, 18, 12) (None, 6, 16, 18, 12)
(None, 6, 43, 6, 4)
(None, 6, 43, 1, 1)
(None, 258)
(None, 128)
(None, 6)
New m222 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 4s 175ms/step - loss: 1.5612 - mse: 0.1238 - accuracy: 0.3769 - precision: 0.7234
Epoch 2/25
22/22 [==============================] - 4s 173ms/step - loss: 1.1240 - mse: 0.0958 - accuracy: 0.5682 - precision: 0.7028
Epoch 3/25
22/22 [==============================] - 4s 174ms/step - loss: 0.9250 - mse: 0.0808 - accuracy: 0.6276 - precision: 0.7618
Epoch 4/25
22/22 [==============================] - 4s 174ms/step - loss: 0.8094 - mse: 0.0706 - accuracy: 0.6810 - precision: 0.7987
Epoch 5/25
22/22 [==============================] - 4s 175ms/step - loss: 0.7234 - mse: 0.0632 - accuracy: 0.7404 - precision: 0.8134
Epoch 6/25
22/22 [==============================] - 4s 173ms/step - loss: 0.6461 - mse: 0.0558 - accuracy: 0.7893 - precision: 0.8421
Epoch 7/25
22/22 [==============================] - 4s 174ms/step - loss: 0.5911 - mse: 0.0515 - accuracy: 0.7864 - precision: 0.8368
Epoch 8/25
22/22 [==============================] - 4s 172ms/s

### Model rs322

In [ ]:
loss,mse,acc,prec = kfold_validation("rs332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"tanh")
results_tanh["rs332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/rs332_25epochs_tanh.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh["rs332"], f)

rs332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
correct
(None, 300)
New rs332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 6s 260ms/step - loss: 1.6369 - output_rs332_loss: 1.4254 - output_spm_rs332_loss: 42.2825 - output_rs332_mse: 0.1147 - output_rs332_accuracy: 0.4050 - output_rs332_precision: 0.7394 - output_spm_rs332_mse: 42.2825 - output_spm_rs332_accuracy: 0.0119
Epoch 2/25
22/22 [==============================] - 6s 260ms/step - loss: 1.2706 - output_rs332_loss: 1.0595 - output_spm_rs332_loss: 42.2317 - output_rs332_mse: 0.0894 - output_rs332_accuracy: 0.5861 - output_rs332_precision: 0.7417 - output_spm_rs332_mse: 42.2317 - output_spm_rs332_accuracy: 0.1039
Epoch 3/25
22/22 [==============================] - 6s 260ms/step - loss: 1.1605 - output_rs332_loss: 0.9498 - output_spm_rs332_loss: 42.1547 - output_rs332_mse: 0.0807 - output_rs332_accuracy: 0.6306 - output_rs332_precision: 0.7667 - output_spm_rs332_mse: 42.1547 - output_spm_rs332_accuracy: 0.2611
Epoch 4/25
22/22 [==============================] - 6s 259ms/step - loss: 1.0676 - output_rs33

### Average of all five

In [ ]:
loss,mse,acc,prec = kfold_validation("ensemble",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,30,1,5,'tanh')
results_tanh["ensemble5"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/ensemble5_30epochs_tanh.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh["ensemble5"], f)

s332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
rs332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/30
22/22 [==============================] - 23s 1s/step - loss: 1.5877 - mse: 0.1263 - accuracy: 0.3680 - precision_5: 0.5714
Epoch 2/30
22/22 [==============================] - 23s 1s/step - loss: 1.1813 - mse: 0.0993 - accuracy: 0.5697 - precision_5: 0.8085
Epoch 3/30
22/22 [==============================] - 23s 1s/step - loss: 1.0093 - mse: 0.0857 - accuracy: 0.6439 - precision_5: 0.8390
Epoch 4/30
22/22 [==============================] - 23s 1s/step - loss: 0.8519 - mse: 0.0722 - accuracy: 0.7270 - precision_5: 0.8880
Epoch 5/30
22/22 [==============================] - 23s 1s/step - loss: 0.7609 - mse: 0.0649 - accuracy: 0.7685 - precision_5: 0.8673
Epoch 6/30
22/22 [==============================] - 23s 1s/step - loss: 0.6714 - mse: 0.0577 - accuracy: 0.7923 - precision_5: 0.9116
Epoch 7/30
22/22 [==============================] - 23s 1s/step - loss: 0.6285 - mse: 0.0546 - accuracy: 0.8027 - precision_5: 0.8942
Epoch 8/30
22/22 [==============================] - 23s 1s/ste

In [ ]:
print(results_tanh)
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/all_tanh_RMSProp_30epochs.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_tanh, f)

{'m332': array([[2.23271036, 1.10481191, 0.89279789, 1.13916254, 1.12104595],
       [0.10517897, 0.07734267, 0.06445088, 0.07845643, 0.06672353],
       [0.61538464, 0.71005917, 0.76923078, 0.70833331, 0.75      ],
       [0.63975155, 0.72222221, 0.77914113, 0.71515149, 0.75449103]]), 'm322': array([[1.50189686, 1.07385194, 1.15622187, 1.1340059 , 1.2498877 ],
       [0.09038183, 0.07583959, 0.08103921, 0.08138732, 0.07879156],
       [0.65680474, 0.69822484, 0.69230771, 0.68452382, 0.6964286 ],
       [0.66463417, 0.71341461, 0.71974522, 0.69512194, 0.71428573]]), 'm222': array([[1.23433244, 1.07053387, 1.28994107, 1.2069459 , 1.59902668],
       [0.06920992, 0.07525548, 0.08288603, 0.07261007, 0.09609405],
       [0.76331359, 0.71005917, 0.69822484, 0.72619045, 0.63690478],
       [0.77439022, 0.71604937, 0.71165645, 0.73456788, 0.65625   ]]), 'rs332': array([[1.45090783, 2.31357837, 1.67474961, 1.37894809, 1.60475481],
       [0.09002029, 0.10770057, 0.08622731, 0.08429853, 0.08961

## Swish and RMSProp

### Definition

In [ ]:
def swish(x):
    return (keras.activations.sigmoid(x) * x)

### Model s332

In [ ]:
results_swish = {}
loss,mse,acc,prec = kfold_validation("s332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"swish")
results_swish["s332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/s332_30epochs_swish.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_swish["s332"], f)

s332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
New s332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 6s 259ms/step - loss: 1.5384 - mse: 0.1232 - accuracy: 0.3961 - precision: 0.6711
Epoch 2/25
22/22 [==============================] - 6s 257ms/step - loss: 1.1724 - mse: 0.0993 - accuracy: 0.5415 - precision: 0.6711
Epoch 3/25
22/22 [==============================] - 6s 257ms/step - loss: 1.0185 - mse: 0.0878 - accuracy: 0.6068 - precision: 0.7039
Epoch 4/25
22/22 [==============================] - 6s 256ms/step - loss: 0.9118 - mse: 0.0794 - accuracy: 0.6513 - precision: 0.7394
Epoch 5/25
22/22 [==============================] - 6s 255ms/step - loss: 0.7975 - mse: 0.0690 - accuracy: 0.7092 - precision: 0.7895
Epoch 6/25
22/22 [==============================] - 6s 256ms/step - loss: 0.7300 - mse: 0.0629 - accuracy: 0.7463 - precision: 0.8225
Epoch 7/25
22/22 [==============================] - 6s 255ms/step - loss: 0.6425 - mse: 0.0545 - accuracy: 0.7967 - precision: 0.8539
Epoch 8/25
22/22 [==============================] - 6s 256ms/s

### Model m332

In [ ]:
loss,mse,acc,prec = kfold_validation("m332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"swish")
results_swish["m332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m332_30epochs_swish.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_swish["m332"], f)

m332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 14, 24, 18) (None, 2, 12, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 12, 18, 12) (None, 6, 10, 18, 12)
(None, 6, 27, 6, 4)
(None, 6, 27, 1, 1)
(None, 162)
(None, 128)
(None, 6)
New m332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 6s 253ms/step - loss: 1.5926 - mse: 0.1259 - accuracy: 0.3501 - precision: 0.7949
Epoch 2/25
22/22 [==============================] - 6s 252ms/step - loss: 1.1745 - mse: 0.0995 - accuracy: 0.5430 - precision: 0.7029
Epoch 3/25
22/22 [==============================] - 6s 252ms/step - loss: 1.0163 - mse: 0.0871 - accuracy: 0.6024 - precision: 0.7440
Epoch 4/25
22/22 [==============================] - 6s 251ms/step - loss: 0.9090 - mse: 0.0783 - accuracy: 0.6647 - precision: 0.7577
Epoch 5/25
22/22 [==============================] - 6s 251ms/step - loss: 0.8303 - mse: 0.0723 - accuracy: 0.6958 - precision: 0.7714
Epoch 6/25
22/22 [==============================] - 6s 253ms/step - loss: 0.7880 - mse: 0.0679 - accuracy: 0.7062 - precision: 0.7561
Epoch 7/25
22/22 [==============================] - 6s 253ms/step - loss: 0.7275 - mse: 0.0635 - accuracy: 0.7255 - precision: 0.8045
Epoch 8/25
22/22 [==============================] - 6s 252ms/s

### Model m322

In [ ]:
loss,mse,acc,prec = kfold_validation("m322",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"swish")
results_swish["m322"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m322_30epochs_swish.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_swish["m322"], f)

m322
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 14, 24, 18) (None, 2, 12, 24, 18)
(None, 6, 7, 18, 12) (None, 6, 14, 18, 12) (None, 6, 12, 18, 12)
(None, 6, 33, 6, 4)
(None, 6, 33, 1, 1)
(None, 198)
(None, 128)
(None, 6)
New m322 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 6s 257ms/step - loss: 1.5901 - mse: 0.1267 - accuracy: 0.3294 - precision: 0.7419
Epoch 2/25
22/22 [==============================] - 5s 246ms/step - loss: 1.1433 - mse: 0.0961 - accuracy: 0.5742 - precision: 0.7167
Epoch 3/25
22/22 [==============================] - 5s 245ms/step - loss: 0.9720 - mse: 0.0825 - accuracy: 0.6454 - precision: 0.7554
Epoch 4/25
22/22 [==============================] - 5s 246ms/step - loss: 0.8536 - mse: 0.0731 - accuracy: 0.6780 - precision: 0.7692
Epoch 5/25
22/22 [==============================] - 5s 246ms/step - loss: 0.7632 - mse: 0.0659 - accuracy: 0.7181 - precision: 0.7829
Epoch 6/25
22/22 [==============================] - 5s 245ms/step - loss: 0.6667 - mse: 0.0578 - accuracy: 0.7596 - precision: 0.8142
Epoch 7/25
22/22 [==============================] - 5s 246ms/step - loss: 0.6346 - mse: 0.0553 - accuracy: 0.7582 - precision: 0.8229
Epoch 8/25
22/22 [==============================] - 5s 244ms/s

NameError: ignored

### Model m222

In [ ]:
loss,mse,acc,prec = kfold_validation("m222",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"swish")
results_swish["m222"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/m222_30epochs_swish.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_swish["m222"], f)

m222
(None, 2, 9, 72, 54) (None, 2, 9, 72, 54) (None, 2, 9, 72, 54) (None, 2, 8, 72, 54) (None, 2, 8, 72, 54)
(None, 2, 9, 24, 18) (None, 2, 18, 24, 18) (None, 2, 16, 24, 18)
(None, 6, 9, 18, 12) (None, 6, 18, 18, 12) (None, 6, 16, 18, 12)
(None, 6, 43, 6, 4)
(None, 6, 43, 1, 1)
(None, 258)
(None, 128)
(None, 6)
New m222 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 4s 191ms/step - loss: 1.6345 - mse: 0.1291 - accuracy: 0.3501 - precision: 0.9000
Epoch 2/25
22/22 [==============================] - 4s 179ms/step - loss: 1.1567 - mse: 0.0956 - accuracy: 0.5653 - precision: 0.7658
Epoch 3/25
22/22 [==============================] - 4s 177ms/step - loss: 0.9627 - mse: 0.0816 - accuracy: 0.6588 - precision: 0.7624
Epoch 4/25
22/22 [==============================] - 4s 178ms/step - loss: 0.8384 - mse: 0.0718 - accuracy: 0.6869 - precision: 0.7966
Epoch 5/25
22/22 [==============================] - 4s 179ms/step - loss: 0.7719 - mse: 0.0663 - accuracy: 0.7240 - precision: 0.8123
Epoch 6/25
22/22 [==============================] - 4s 178ms/step - loss: 0.7075 - mse: 0.0601 - accuracy: 0.7567 - precision: 0.8299
Epoch 7/25
22/22 [==============================] - 4s 177ms/step - loss: 0.6469 - mse: 0.0556 - accuracy: 0.7700 - precision: 0.8471
Epoch 8/25
22/22 [==============================] - 4s 177ms/s

In [ ]:
loss,mse,acc,prec = kfold_validation("rs332",M1_ufc,M2_ufc,M3_ufc,M4_ufc,M5_ufc,Train_ufc_y,pca_SPM_cube_ufc_reloaded,32,25,1,None,"swish")
results_swish["rs332"] = np.array((loss,mse,acc,prec))
filename = '/content/drive/MyDrive/ML_CSP774/Theory_project/3D-CNN Activity Recognition/Pickle/rs332_30epochs_swish.pickle'
with open(filename, 'wb') as f:
    pickle.dump(results_swish["rs332"], f)

rs332
(None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 7, 72, 54) (None, 2, 6, 72, 54) (None, 2, 6, 72, 54)
(None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 7, 24, 18) (None, 2, 6, 24, 18) (None, 2, 6, 24, 18)
(None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 5, 18, 12) (None, 6, 4, 18, 12) (None, 6, 4, 18, 12)
(None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 5, 6, 4) (None, 6, 4, 6, 4) (None, 6, 4, 6, 4)
(None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 5, 1, 1) (None, 6, 4, 1, 1) (None, 6, 4, 1, 1)
(None, 6, 23, 1, 1)
(None, 138)
(None, 128)
(None, 6)
correct
(None, 300)
New rs332 model initialised
------------------------------------------------------------------------
Training for fold 1 ...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/25
22/22 [==============================] - 6s 264ms/step - loss: 1.7718 - output_rs332_loss: 1.5676 - output_spm_rs332_loss: 40.8470 - output_rs332_mse: 0.1253 - output_rs332_accuracy: 0.3665 - output_rs332_precision_1: 0.7027 - output_spm_rs332_mse: 40.8470 - output_spm_rs332_accuracy: 0.0074
Epoch 2/25
22/22 [==============================] - 6s 264ms/step - loss: 1.3883 - output_rs332_loss: 1.1842 - output_spm_rs332_loss: 40.8190 - output_rs332_mse: 0.0998 - output_rs332_accuracy: 0.5223 - output_rs332_precision_1: 0.7137 - output_spm_rs332_mse: 40.8190 - output_spm_rs332_accuracy: 0.1306
Epoch 3/25
22/22 [==============================] - 6s 263ms/step - loss: 1.2029 - output_rs332_loss: 0.9991 - output_spm_rs332_loss: 40.7634 - output_rs332_mse: 0.0852 - output_rs332_accuracy: 0.6142 - output_rs332_precision_1: 0.7338 - output_spm_rs332_mse: 40.7634 - output_spm_rs332_accuracy: 0.2062
Epoch 4/25
22/22 [==============================] - 6s 264ms/step - loss: 1.0569 - outpu

# Completed